# Rustの言語機能

## 非同期処理

Rust 1.39 より `async` と `await` という非同期プログラミング用のキーワードが導入されている

これは JavaScript (ECMAScript) で導入されている async, await とほぼ同様の機能と考えて良い

JavaScriptの async, await は Promise という概念を簡単に扱えるようにしたものだが、Rustも似たようなもので `Future` という概念を簡単に扱えるようにしたものである

例えば、データベースにアクセスしてデータを受け取りたい場合、Rustでは以下のような非同期プログラミングを行い、パフォーマンスの良い I/O を実現できる

```rust
// $ cargo add futures

use futures::executor;

struct User {
    // ... Some data
}

struct UserId(u32);

struct Db {};

impl Db {
    async fn find_by_user_id(&self, user_id: UserId) -> Option<User> {
        // ... Db接続処理等の実装
    }
}

async fn find_user_by_id(db: Db, user_id: UserId) -> Option<User> {
    db.find_by_user_id(user_id).await
}

fn main() {
    // find_by_user_id 非同期関数の呼び出し
    executor::block_on(find_user_by_id(Db{}, UserId(1)));
}
```

### 非同期プログラミングとは
非同期処理とは一般に「あるタスクの実行中に、別のタスクを実行できること」という並列処理のことを指す

その実現方法には、これまでマルチプロセスやマルチスレッドがあり、Rustにおいても前項で示したような `thread::spawn` などの関数が用意されている

しかしながら、マルチプロセスやマルチスレッドには **C10K問題** と呼ばれる問題が存在し、同時に走るプロセス・スレッドが万単位に増えると応答性能が極端に悪くなってしまう

これは、表現できるプロセス番号の制限や、OSスレッド1つあたりが利用するメモリ量の制限によって引き起こされる問題であり、Webサービスが大規模化してきた2000年代後半頃から顕在化してきた

この問題を解決するため、プロセス数・スレッド数は変えず「必要なタイミングに、必要な処理を、空いているリソースに割り当てて実行する」というイベント駆動モデルの実装が登場し普及している

この方式では、入力待ちや通信応答待ち、タイマーによるインターバルに他の処理を割り当てて、限られたリソース上で効率よく動作する

一方、処理単位（タスク・ジョブ）の割り当てをランタイム上で行うために複雑な実装が行われている

ここでは、このイベント駆動モデル型の並列処理機構のことを非同期処理と呼ぶことにする

非同期プログラミングとはすなわち、非同期タスクを作成してランタイムに実行させる形式のプログラミングスタイルである

前述の通り、非同期処理ランタイムの中身は非常に複雑な実装となっているが、プログラマ側はその詳細を気にせず、簡易な記述で実装できるように工夫がなされてきた

その代表例が async/await で、C#が発祥であるが、今ではJavaScriptなど他の言語でも採用され、Rustにも 1.39.0 で採用されている

### Future
Rustでは、async/await の裏にある Future の概念を理解しておくことで、より非同期処理の動作に理解が深まる

Futureとは、ざっくり言えば「将来のどこかの時点で処理が完了するタスク」である

Rustにおける用語の関係性をおおよそで言い表すと以下のような形となる

- Future:
    - 非同期タスクのこと
    - Futureトレイトを実装した型を戻り値とすることで、非同期ランタイムに渡った時点で処理タイミングが制御されるようになる
- 非同期ランタイム:
    - Futureの実行タイミングを制御し、必要なタイミングで処理を走らせるシステムのこと
    - Rustで単に「ランタイム」というときは非同期ランタイムのことを言っている場合が多い
- async:
    - 「Futureトレイトを実装した型を返す関数」を通常の関数のように記述できるようにする構文
- await:
    - asyncで記述された関数の処理結果を受け取る必要があるタイミングを記述する構文

`std::future::Future` トレイトは以下のような実装となっており、`Poll<Self::Output>` を返す `poll` メソッドのみを定義されている

```rust
pub trait Future {
    type Output;
    fn poll(self: Pin<&mut Self>, cx: &mut Context) -> Poll<Self::Output>;
}
```

`Poll<T>` とは `Ready(T)` または `Pending` の2値を持つ enum である

Futureトレイトを実装したタスクが「将来的に値が確定する計算」となり、タスクが作成された時点では実行されず、ランタイムに乗った時点でスケジューリングされて実行される

その実行するかどうかの判断に使われるメソッドが `poll` である

実行するかどうかチェックする主体は `std::walker::Waker` であり、pollメソッドの引数に渡される `Context` 内にラップされている

pollメソッドが `Poll::Pending` を返した場合、また他のタスクが完了するまでpollは呼ばれず、他のタスクを実行する

そして `Poll::Ready(T)` が返されると、タスクが実行完了となり、ランタイムは次のタスクを実行状態に変更する

このポーリング作業を繰り返しながら実行するランタイムのことを `executor` と呼ぶ

各Futureはステートマシンとして表現されており、一箇所のヒープに割り当てられる

ステートマシンの中には、IOイベントごとに一つの状態が格納されており、各タスクに対して決まったメモリサイズを1箇所にしか割り当てないため、余計なメモリ割り当てのコストをかけず、ゼロコスト抽象化を実現している

In [2]:
:dep futures
// => $ cargo add futures

use futures::{executor, future::join_all};
use std::future::Future;
use std::pin::Pin;
use std::task::{Context, Poll};

// (u32, String) タプル型の CountDown構造体
struct CountDown(u32, String);

// CountDown構造体を Future 実装にする
impl Future for CountDown {
    type Output = String; // アウトプットを文字列型に設定
    
    // 非同期タスクの実行判定メソッドを定義
    fn poll(mut self: Pin<&mut Self>, cx: &mut Context) -> Poll<String> {
        // CountDown が 0 になっていればタスク完了: 次のタスクの実行準備OK
        if self.0 == 0 {
            return Poll::Ready(format!("[{}] count zero!!", self.1).to_string());
        }
        // タスク実行中: CountDown のタプル値をデクリメント
        println!("[{}] count: {}", self.1, self.0);
        self.0 -= 1;
        cx.waker().wake_by_ref(); // Waker に再度呼んでもらうことを依頼
        return Poll::Pending; // 次のタスクはまだペンディング
    }
}

fn main() {
    // CountDownタスクを2つ作成
    let countdown_future1 = CountDown(10, "timer1".to_string());
    let countdown_future2 = CountDown(20, "timer2".to_string());
    
    // 作成したCountDownタスクを一つにまとめて、executore ランタイムに乗せる
    let task_set = join_all(vec![countdown_future1, countdown_future2]);
    let result = executor::block_on(task_set); // 渡したFutureが完了になるまでブロックして待機
    
    // 各タスクの実行結果を取得
    for (index, output) in result.iter().enumerate() {
        println!("{}: {}", index, output);
    }
}
main();

[timer1] count: 10
[timer2] count: 20
[timer1] count: 9
[timer2] count: 19
[timer1] count: 8
[timer2] count: 18
[timer1] count: 7


### async, await
async/await は非同期処理を制御するためのキーワードで、内部的にはジェネレータとして表現されている

ジェネレータとは、一旦区切りの良いところまで処理を行って中断したあと、もう一度準備が整った段階で処理を再開できる機構である

これはステートマシンになっており、中断と再開の状態を切り替えながら徐々に処理を進行させる

ジェネレータを用いることにより、複数の処理を切り替えながら効率よく処理を完遂できる

In [3]:
:dep futures
// => $ cargo add futures

// 2つの値を加算する関数を async で記述
async fn async_add(left: i32, right: i32) -> i32 {
    left + right
}

// async_add を呼び出す関数
// ※ .await は async fn 関数ブロックの中でしか呼び出せない
async fn something_great_async_function() -> i32 {
    let ans = async_add(2, 3).await; // この時点で処理結果 5 を取り出せる
    println!("{}", ans);
    ans
}

fn main() {
    // async/await を実行するためにはランタイムが必要
    // ここでは futures::executor ランタイムを使う
    executor::block_on(something_great_async_function());
}
main();

[timer2] count: 17
[timer1] count: 6
[timer2] count: 16
[timer1] count: 5
[timer2] count: 15
[timer1] count: 4
[timer2] count: 14
[timer1] count: 3
[timer2] count: 13
[timer1] count: 2
[timer2] count: 12
[timer1] count: 1
[timer2] count: 11
[timer2] count: 10
[timer2] count: 9
[timer2] count: 8
[timer2] count: 7
[timer2] count: 6
[timer2] count: 5
[timer2] count: 4
[timer2] count: 3
[timer2] count: 2
[timer2] count: 1
0: [timer1] count zero!!
1: [timer2] count zero!!
5


In [4]:
/*
    async fn は impl Future のシンタックスシュガーになっている
    よって、以下のように Future を直接記述しても同じである
    ただし、処理の見通しのことを考えると async/await を使うほうがコードが見やすい
*/
fn something_great_async_function() -> impl Future<Output = i32> {
    async {
        let ans = async_add(2, 3).await;
        println!("{}", ans);
        ans
    }
}

use futures::executor;

fn main() {
    let result = executor::block_on(something_great_async_function());
    dbg!(result);
}
main();

5


[src/lib.rs:48] result = 5


async関数／ブロック内に複数の `.await` が存在した場合には、`.await` ごとに値を受け取る

非同期関数内では `.await` まで逐一処理を進めながらタスクを切り替えていくことになる

async構文では、その都度 await しない限り、中身の評価が走らないため要注意である

#### ムーブ
通常、asyncブロックが始まった時点でそれ以前のスコープは外れ、変数はライフタイムを終える

そのため、asyncブロック内で外側のスコープの変数を使用したい場合は、所有権を移動するために `move` キーワードを使用することができる

In [5]:
fn move_to_async_block() -> impl Future<Output = ()> {
    // 文字列を所有する変数を定義
    let outside_var = String::from("this is outside");
    // 通常なら、ここで outside_var の所有権がなくなるため、asyncブロック内で使うことはできない
    // => move で所有権を async ブロック内に移動することで使えるようになる
    async move {
        println!("{}", outside_var);
    }
}

executor::block_on(move_to_async_block());

this is outside


#### ライフタイム
通常の非staticなライフタイムであれば、async内におけるライフタイムはほとんど問題にならない

例えば、ある引数を持つ関数の Future のライフタイムは次のようになっている

```rust
// ある引数を持つ非同期関数
async fn some_great_function(arg: &i32) -> i32 {
    *arg
}

// => 上記関数は、コンパイラにより以下のような関数に展開される
/// ライフタイム 'a を持ち、戻り値が Future である関数
fn some_great_function<'a>(arg: &'a i32) -> impl Future<Output = i32> + 'a {
    async move {
        *arg
    }
}
```

しかし、スレッドをまたいで Future の値を送りたくなった場合は `'static` ライフタイムを用いる必要がある

```rust
fn some_great_function() -> impl Future<Output = i32> {
    let value: i32 = 5;
    // value の所有権はここで終わり (次のコードが async 関数／ブロックであるため)
    // 以下のコードは、所有権のない変数参照を渡そうとしているためコンパイルエラー
    send_to_another_thread_with_borrowing(&value);
}

async fn send_to_another_thread_with_borrowing(x: &i32) -> i32 {
    // ...別スレッドへ変数を送る処理
}
```

上記問題を解決するためには、asyncブロック内に `value` 変数を宣言すれば良い

```rust
fn some_great_function() -> impl Future<Output = i32> {
    async {
        let value: i32 = 5;
        send_to_another_thread_with_borrowing(&value);
    }
}
```

### 非同期ランタイム
非同期ランタイムは、非同期計算の実行環境を指す

前述した `futures::executor` 以外にも、以下のようなクレートがよく使われる

- [tokio](https://tokio.rs/)
    - Rustを古くから支えてきた非同期ライブラリ
    - 現在でも多くのライブラリが tokio に依存した実装となっている
- [async-std](https://async.rs/)
    - 近年登場してきた非同期ライブラリ
    - tokio の内部実装やAPIがやや複雑であるという問題を解消するために発足した
    - 利用事例はまだ少なく、今後に期待されるクレート

### async-trait
Rustでは現状、トレイトの関数に async を付けることはできない

したがって、以下のようなコードはエラーとなる

In [6]:
trait AsyncTrait {
    async fn f(&self) {
        println!("trait with async method");
    }
}

Error: functions in traits cannot be declared `async`

この問題は `async-trait` クレートを使用することで回避できる

In [7]:
:dep async-trait
// => $ cargo add async-trait

use async_trait::async_trait;

// トレイトに #[async_trait] attrivute を付けることで async fn を使えるようになる
#[async_trait]
trait AsyncTrait {
    async fn f(&self) {
        println!("trait with async method");
    }
}